In [8]:
import numpy as np
import pandas as pd

# Set a seed so that the random numbers will be reproducible
np.random.seed(12)

# Generate attributes x1, x2 for each of numalts (J) alternatives

In [9]:
# For now, J << 1000 alternatives to speed up runtimes
numalts = 50

def rand(len, min, max):
    """ Generate `len` random floats uniformly distributed from `min` to `max` """
    return (max - min) * np.random.rand(len) + min

# Attribute x is uniformly distributed over [-2, 1] for half the alternatives
# and over [-1, 2] for the other half, as in Guevara & Ben-Akiva

# X = np.concatenate((rand(numalts/2, -2, 1), rand(numalts/2, -1, 2)))

# Or, attribute x is uniformly distributed over [0, 10] for half the alternatives
# and over [100, 110] for the other half, to induce bias in estimation

X = np.concatenate((rand(int(numalts/2), 0, 10), rand(int(numalts/2), 100, 110)))

In [10]:
X

array([1.54162842e+00, 7.40049697e+00, 2.63315015e+00, 5.33739393e+00,
       1.45749625e-01, 9.18747008e+00, 9.00714854e+00, 3.34214276e-01,
       9.56949336e+00, 1.37209321e+00, 2.83828353e+00, 6.06083184e+00,
       9.44225136e+00, 8.52735541e+00, 2.25923352e-02, 5.21226027e+00,
       5.52037633e+00, 4.85377414e+00, 7.68134154e+00, 1.60716753e+00,
       7.64560450e+00, 2.08097980e-01, 1.35210178e+00, 1.16273017e+00,
       3.09897584e+00, 1.06714526e+02, 1.04712298e+02, 1.08161683e+02,
       1.02895868e+02, 1.07331260e+02, 1.07026224e+02, 1.03275695e+02,
       1.03346475e+02, 1.09780581e+02, 1.06245821e+02, 1.09503135e+02,
       1.07674757e+02, 1.08250093e+02, 1.04066403e+02, 1.04513084e+02,
       1.04006316e+02, 1.09951382e+02, 1.01775642e+02, 1.09625969e+02,
       1.04192503e+02, 1.04240524e+02, 1.04631489e+02, 1.03737231e+02,
       1.04655081e+02, 1.00351683e+02])

In [11]:
print(pd.DataFrame(X[:int(numalts/2)]).describe())
print(pd.DataFrame(X[int(numalts/2):]).describe())

               0
count  25.000000
mean    4.470503
std     3.371393
min     0.022592
25%     1.372093
50%     4.853774
75%     7.645605
max     9.569493
                0
count   25.000000
mean   105.626629
std      2.648681
min    100.351683
25%    104.006316
50%    104.655081
75%    107.674757
max    109.951382


# Generate taste coefficient beta for each of numobs (N) agents

In [12]:
# For regular MNL, use a single value instead of a distribution as 
# Guevara & Ben-Akiva used for the mixture model

numobs = 1000  # agents/observations

beta = np.zeros(1000) + 1.5
# beta = 0.8 * np.random.randn(numobs) + 1.5

In [13]:
pd.DataFrame(beta).describe()

,0
count,1000.0
mean,1.5
std,0.0
min,1.5
25%,1.5
50%,1.5
75%,1.5
max,1.5


# Simulate a choice from numalts (J) alternatives for each of numobs (N) agents

In [14]:
# Generate a utility matrix for N agents choosing among J alternatives

U = [[beta[n]*x + np.random.gumbel() for x in X] for n in range(numobs)]
   
len(U), len(U[0])

(1000, 50)

In [41]:
len(U)

1000

In [15]:
# Each agent chooses the alternative with highest utility

choices = [np.argmax(a) for a in U]

len(choices), choices[:10]

(1000, [41, 35, 41, 33, 41, 35, 33, 27, 41, 33])

# 2. Estimate beta without sampling, using PyLogit MNL

In [16]:
import pylogit
from collections import OrderedDict

In [17]:
# Set up the estimation dataset in long format

d = [[n, i, int(choices[n]==i), X[i]] for n in range(numobs) for i in range(numalts)]
df = pd.DataFrame(d, columns=['obs_id', 'alt_id', 'chosen', 'x'])

In [43]:
df.head()

,obs_id,alt_id,chosen,x
0,0,0,0,1.541628
1,0,7,0,0.334214
2,0,8,0,9.569493
3,0,12,0,9.442251
4,0,14,0,0.022592


In [44]:
df.alt_id.describe()

22    403
3     388
19    378
11    377
16    376
15    371
14    371
6     371
24    366
9     364
2     363
20    362
21    360
23    359
10    357
0     356
1     355
4     354
12    354
18    352
41    346
7     346
13    339
8     335
5     332
17    311
33    255
43    190
35    143
27     29
37     16
36     12
25      3
30      3
29      2
34      1
Name: alt_id, dtype: int64

In [19]:
df.describe()

,obs_id,alt_id,chosen,x
count,50000.000000,50000.000000,50000.000000,50000.000000
mean,499.500000,24.500000,0.020000,55.048566
std,288.677877,14.431014,0.140001,50.665719
min,0.000000,0.000000,0.000000,0.022592
25%,249.750000,12.000000,0.000000,4.853774
50%,499.500000,24.500000,0.000000,54.960588
75%,749.250000,37.000000,0.000000,104.655081
max,999.000000,49.000000,1.000000,109.951382


In [20]:
# Set up reusable model spec

spec = OrderedDict([('x', 'all_same')])
labels = OrderedDict([('x', 'beta_x')])

In [21]:
# Set up reusable code to estimate a model

def estimate_model(init_val):
    """
    Initialize and fit a model, returning it as an object. Will use the 
    current values of `df`, `spec`, and `labels`.
    """
    m = pylogit.create_choice_model(data = df, 
                                    alt_id_col = 'alt_id', 
                                    obs_id_col = 'obs_id', 
                                    choice_col = 'chosen', 
                                    specification = spec, 
                                    model_type = "MNL", 
                                    names = labels)

    m.fit_mle(init_vals = np.array([init_val]))
    return m

In [23]:
%%time
m = estimate_model(init_val = 1.2)
m.get_statsmodels_summary()

Log-likelihood at zero: -3,912.0230
Initial Log-likelihood: -1,618.0274
Estimation Time for Point Estimation: 0.05 seconds.
Final log-likelihood: -1,592.6368


/home/danph/anaconda3/envs/dev/lib/python3.8/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


CPU times: user 744 ms, sys: 338 ms, total: 1.08 s
Wall time: 825 ms


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      chosen   No. Observations:                1,000
Model:             Multinomial Logit Model   Df Residuals:                      999
Method:                                MLE   Df Model:                            1
Date:                     Wed, 09 Jun 2021   Pseudo R-squ.:                   0.593
Time:                             15:17:22   Pseudo R-bar-squ.:               0.593
AIC:                             3,187.274   Log-Likelihood:             -1,592.637
BIC:                             3,192.181   LL-Null:                    -3,912.023
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
beta_x         1.5995      0.064     24.891      0.000       1.474       1.725
==============================================================================
"""

# 3a. Estimate beta with random sampling of alternatives

In [24]:
# In the estimation dataset, for each observation include a row for the
# chosen alternative, plus K-1 other alternatives sampled randomly
# without replacement, where K < J.

# Some more notation:
# - true choice set C = range(J)
# - restricted choice set D_n is a subset of C, where len(D_n) = K

In [26]:
# TO DO - rewrite to use sampling weights

def alts(obs_id, C, K):
    """
    This function generates a restricted choice set D for a particular
    observation. Expects list `C` of alternatives to sample from (either
    the full choice set or a stratrum), int `K` alternatives to sample,
    and list `choices` of the alt_id chosen for each obs_id. Returns list 
    of K alt_id's including the chosen one.
    """
    chosen = choices[obs_id]  # id of chosen alternative
    unchosen = [i for i in C if chosen != i]  # id's of unchosen alts
    sample_unchosen = np.random.choice(unchosen, size=K-1, replace=False).tolist()
    return np.sort([chosen] + sample_unchosen)
    
print(alts(0, range(numalts), 5))

[18 20 27 41 49]


In [27]:
# Set up the estimation dataset, which can use the same spec as earlier

C = range(numalts)  # choice set to sample from
K = 10

d = [[n, i, int(choices[n]==i), X[i]] for n in range(numobs) for i in alts(n, C, K)]
df = pd.DataFrame(d, columns=['obs_id', 'alt_id', 'chosen', 'x'])

In [28]:
df.head()

,obs_id,alt_id,chosen,x
0,0,7,0,0.334214
1,0,18,0,7.681342
2,0,20,0,7.645605
3,0,26,0,104.712298
4,0,27,0,108.161683


In [30]:
df.shape

(10000, 4)

In [29]:
df.describe()

,obs_id,alt_id,chosen,x
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,499.500000,25.394400,0.100000,58.988731
std,288.689425,14.355405,0.300015,50.836763
min,0.000000,0.000000,0.000000,0.022592
25%,249.750000,13.000000,0.000000,5.212260
50%,499.500000,26.000000,0.000000,101.775642
75%,749.250000,38.000000,0.000000,106.714526
max,999.000000,49.000000,1.000000,109.951382


In [31]:
%%time
m = estimate_model(init_val = 1.2)
m.get_statsmodels_summary()

Log-likelihood at zero: -2,302.5851
Initial Log-likelihood: -469.0747
Estimation Time for Point Estimation: 0.02 seconds.
Final log-likelihood: -449.9743
CPU times: user 255 ms, sys: 113 ms, total: 368 ms
Wall time: 193 ms


/home/danph/anaconda3/envs/dev/lib/python3.8/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      chosen   No. Observations:                1,000
Model:             Multinomial Logit Model   Df Residuals:                      999
Method:                                MLE   Df Model:                            1
Date:                     Wed, 09 Jun 2021   Pseudo R-squ.:                   0.805
Time:                             15:19:12   Pseudo R-bar-squ.:               0.804
AIC:                               901.949   Log-Likelihood:               -449.974
BIC:                               906.856   LL-Null:                    -2,302.585
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
beta_x         1.6864      0.094     18.016      0.000       1.503       1.870
==============================================================================
"""

# Run 1000x with different samples of alternatives

In [32]:
%%time
%%capture

beta = []
C = range(numalts)
K = 10

for i in range(100):
    d = [[n, i, int(choices[n]==i), X[i]] for n in range(numobs) for i in alts(n, C, K)]
    df = pd.DataFrame(d, columns=['obs_id', 'alt_id', 'chosen', 'x'])
    m = estimate_model(init_val = 1.2)
    beta.append(m.params.beta_x)

CPU times: user 45.1 s, sys: 9.83 s, total: 54.9 s
Wall time: 34.8 s


In [33]:
pd.Series(beta).describe()
# Looks unbiased, as expected. It's very close to the true beta of 1.5

count    100.000000
mean       1.594502
std        0.058690
min        1.458312
25%        1.553212
50%        1.592828
75%        1.631565
max        1.750869
dtype: float64

# 3b. Estimate beta with over-sampling of irrelevant alternatives

In [35]:
# Recall that half the values of x are in the range [0, 10] and half are
# in the range [100, 110]. The taste coefficient is positive, so the first
# set of alternatives is much less relevant than the second set. 

C = range(int(numalts/2))  # alternatives to sample from
K = 10

d = [[n, i, int(choices[n]==i), X[i]] for n in range(numobs) for i in alts(n, C, K)]
df = pd.DataFrame(d, columns=['obs_id', 'alt_id', 'chosen', 'x'])

In [36]:
df.head()

,obs_id,alt_id,chosen,x
0,0,0,0,1.541628
1,0,7,0,0.334214
2,0,8,0,9.569493
3,0,12,0,9.442251
4,0,14,0,0.022592


In [37]:
df.describe()

,obs_id,alt_id,chosen,x
count,10000.000000,10000.00000,10000.000000,10000.000000
mean,499.500000,14.64240,0.100000,14.942447
std,288.689425,10.44116,0.300015,31.723855
min,0.000000,0.00000,0.000000,0.022592
25%,249.750000,6.00000,0.000000,1.372093
50%,499.500000,14.00000,0.000000,5.212260
75%,749.250000,20.00000,0.000000,8.527355
max,999.000000,43.00000,1.000000,109.951382


In [38]:
%%time
m = estimate_model(init_val = 1.5)
m.get_statsmodels_summary()

Log-likelihood at zero: -2,302.5851
Initial Log-likelihood: 0.0000
Estimation Time for Point Estimation: 0.00 seconds.
Final log-likelihood: 0.0000
CPU times: user 291 ms, sys: 82.3 ms, total: 373 ms
Wall time: 198 ms


/home/danph/anaconda3/envs/dev/lib/python3.8/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      chosen   No. Observations:                1,000
Model:             Multinomial Logit Model   Df Residuals:                      999
Method:                                MLE   Df Model:                            1
Date:                     Wed, 09 Jun 2021   Pseudo R-squ.:                   1.000
Time:                             15:21:10   Pseudo R-bar-squ.:               1.000
AIC:                                 2.000   Log-Likelihood:                  0.000
BIC:                                 6.908   LL-Null:                    -2,302.585
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
beta_x         1.5000   2.32e+04   6.47e-05      1.000   -4.54e+04    4.54e+04
==============================================================================
"""

# 5. MNL with sampling correction

Utility of alternative j: $$ V_{j} = \beta x_{j} $$

With sampling, we have to account for the restricted choice set (from Eq 6 in Guevara & Ben-Akiva 2013):

$$ V_j = \beta x_j + \ln \pi(D \mid j) $$
Where pi is the conditional probability that we would construct the choice set D given that alternative j was chosen. This goes into the likelihood function in both the numerator and denominator.

$$ L_n = \frac {exp(\beta x_i + \ln \pi(D_n \mid i))} {\sum_{j \epsilon D_n} exp(\beta x_j + \ln \pi(D_n \mid j))} $$
How to calculate pi? From the original formulation of this in McFadden 1978: "Suppose D is comprized of i plus a sample of alternatives from the set C\{i}, obtained by considering each element of this set independently, and including it with probability p. Then, the probability of D will depend solely on the number of elements K it contains."

$$ \pi(D) = p^{K-1} (1 - p)^{J-K} $$
(?? Without replacement, i think it should be the n-choose-k binomial coefficient, where n=J-1 and k=K-1)

$$ \pi(D) = {n \choose k} = \frac {(K-1)!(J-K)!} {(J-1)!} $$

In [40]:
# Add a column in the estimation data for the constant
N = 1000
d = [[n, i, int(C[n]==i), X[i], 1] for n in range(N) for i in alts(n)]

df = pd.DataFrame(d, columns=['obs_id', 'alt_id', 'choice', 'x', 'const'])

TypeError: alts() missing 2 required positional arguments: 'C' and 'K'